# Imports

##### General imports

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import copy

##### Import from flatland environment 

In [ ]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator
from flatland.utils.rendertools import RenderTool

##### Import from our framework

In [ ]:
from src.graph import NetworkGraph
from src.flows import TimeNetwork
from src.flows import MCFlow

# Test of time expanded network

##### Create a flatland network

In [ ]:
number_agents = 10

size_side = 20
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=complex_rail_generator(nr_start_goal=20, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, 
                                                    seed = np.random.randint(0,2000)),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_agents,
              obs_builder_object=GlobalObsForRailEnv())

env.reset()

env_renderer = RenderTool(env)
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)


matrix_rail = np.array(env.rail.grid.tolist())
flatlandNetwork = NetworkGraph(matrix_rail,[(0,1)],[(1,0)])

In [ ]:
sources = []
sinks = []
for agent in env.agents:
    sources.append(agent.initial_position)
    sinks.append(agent.target)

##### create a time expanded network

In [ ]:
import time

In [ ]:
start = time.time()
TestNetworkTime = TimeNetwork(flatlandNetwork, depth=50)
stop = time.time()
print(f'time taken to build the graph: {stop-start}')

In [ ]:
TestNetworkTime.connect_sources_and_sink(sources,sinks)

##### Test LP Formulation

test a simple graph

In [ ]:
mcflow = MCFlow(TestNetworkTime.graph,len(sources),TestNetworkTime.get_topology_network())

In [ ]:
mcflow.solve()

In [ ]:
paths = mcflow.get_paths_solution()

In [ ]:
lengths = []
pathsToAllongate = copy.deepcopy(mcflow.solution)
for agent,path in pathsToAllongate.items():
    lengths.append(len(path))
    
maxLength = max(lengths)
for agent,path in pathsToAllongate.items():
    for i in range(maxLength-len(path)):
        path.append(None)
    pathsToAllongate[agent] = path

In [ ]:
dfPaths = pd.DataFrame(pathsToAllongate)
dfPaths